In [1]:
import pandas as pd
for encoding in ['utf-8', 'latin-1', 'windows-1252']:
    try:
        df = pd.read_csv('./sales_data_sample.csv', encoding=encoding)
        df['POSTALCODE'] = df['POSTALCODE'].fillna(0000)
        df[['QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER', 'SALES']] = df[['QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER', 'SALES']].fillna(0) 
        df[['STATUS', 'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE',  "COUNTRY", "TERRITORY", "CONTACTLASTNAME","CONTACTFIRSTNAME", "DEALSIZE" ]] = df[['STATUS', 'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE',  "COUNTRY", "TERRITORY", "CONTACTLASTNAME","CONTACTFIRSTNAME", "DEALSIZE" ]].fillna('N/A') 
        break 
    except UnicodeDecodeError:
        pass

In [11]:
from flask import Flask, jsonify, request
app = Flask(__name__)

selected_features = ["MONTH_ID", "YEAR_ID", "PRODUCTLINE", "SALES", "QUANTITYORDERED"]
df1 = df[selected_features]

@app.route("/trends", methods=['POST'])
def get_recommendation():
  user_ask = request.json
  month = user_ask['month']

  monthly_data = df[df["MONTH_ID"] == int(month)]
  motorcycle_sales = monthly_data[monthly_data["PRODUCTLINE"] == "Motorcycles"]["SALES"]
  avg_motorcycle_sales = motorcycle_sales.mean()


  previous_month_data = df[df["MONTH_ID"] == int(month) - 1]
  previous_motorcycle_sales = previous_month_data[previous_month_data["PRODUCTLINE"] == "Motorcycles"]["SALES"].mean()
  
  print(previous_motorcycle_sales)
  trend_message = ""
  
  if avg_motorcycle_sales > previous_motorcycle_sales:
    trend_message = "There's a potential upward trend in Motorcycle sales."
  else:
    trend_message = "There's a potential downward trend in Motorcycle sales."

  recommendation = trend_message 

  return jsonify({"recommendation": recommendation})

if __name__ == "__main__":
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [32]:
#SQL 1, What is my top earning sale item?
top_earning_item = df.groupby('PRODUCTLINE')['SALES'].sum().sort_values(ascending=False).index[0]
print(top_earning_item)

Classic Cars


In [33]:
#SQL 2, CITY with best sells?
best_sales_city = df.groupby('CITY')['SALES'].sum().sort_values(ascending=False).index[0]
print(f"City with Best Sales: {best_sales_city}")

City with Best Sales: Madrid


In [51]:
# SQL 3 Sales Performance Analysis
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])
filtered_df = df[(df['ORDERDATE'].dt.year == 2003) & (df['QUANTITYORDERED'] >= 40) & (df['STATUS'] == 'Shipped') & (df['ORDERDATE'].dt.quarter == 4)]
top_products = filtered_df.groupby('PRODUCTLINE')['SALES'].sum().sort_values(ascending=False).head(5).index.tolist()
print(f"Top 5 Products): {top_products}")

Top 5 Products (Q4 2003, Shipped, Min 40 Units): ['Classic Cars', 'Vintage Cars', 'Trucks and Buses', 'Motorcycles', 'Planes']


In [80]:
# SQL 4 Customer Segmentation Query
filtered_df = df[(df['ORDERDATE'].dt.year == 2003) & (df['SALES'] > 5000) & (df['QUANTITYORDERED'] > 3) & (df['COUNTRY'].isin(['USA', 'France']))]
print("Customers:")
print(filtered_df[['CUSTOMERNAME', 'QUANTITYORDERED']])


Customers (3+ orders > $5000, USA/France):
                      CUSTOMERNAME  QUANTITYORDERED
4         Corporate Gift Ideas Co.               49
29            Technics Stores Inc.               37
33         Classic Gift Ideas, Inc               23
34          Saveley & Henriot, Co.               42
57               Toys4GrownUps.com               37
...                            ...              ...
1977  Online Diecast Creations Co.               45
2047          Classic Legends Inc.               44
2079       Classic Gift Ideas, Inc               47
2380            Muscle Machine Inc               46
2386                 Mini Classics               49

[92 rows x 2 columns]


In [113]:
# SQL 5 Product Demand Fluctuation
filtered_df = df[(df['ORDERDATE'].dt.year == 2003) & (df['PRICEEACH'] > 80) & (df['SALES'] > 10)]
if(len(filtered_df)):
    monthly_avg_quantity = filtered_df.groupby([df['ORDERDATE'].dt.month])['QUANTITYORDERED'].mean()
    highest_demand_month = monthly_avg_quantity
    print(f"Month with Highest Avg Order Quantity (index 1-12 = jan-dec): {highest_demand_month}")
else:
    print(f"No results found")

Month with Highest Avg Order Quantity (index 1-12 = jan-dec): ORDERDATE
1     31.520000
2     33.280000
3     35.645161
4     34.500000
5     34.142857
6     35.900000
7     35.028571
8     34.829268
9     32.666667
10    35.170000
11    34.401070
12    36.437500
Name: QUANTITYORDERED, dtype: float64


In [125]:
#SQL 6, regional state comparison
filtered_df = df[(df['ORDERDATE'].dt.year == 2003) & (df['STATUS'] == 'Shipped') & (df['QUANTITYORDERED'] >= 20)]

state1_avg_order_value = filtered_df[filtered_df['STATE'] == 'NSW']['SALES'].mean()
state2_avg_order_value = filtered_df[filtered_df['STATE'] == 'CA']['SALES'].mean()

print(f"Average Order Value Comparison:")
print(state1_avg_order_value)
print(state2_avg_order_value)


Average Order Value Comparison:
3353.742
3442.921717791411


In [145]:
#SQL 7: Order Fulfillment Efficiency: For orders placed in the first half of 2003, 
# find out which country had the highest proportion of orders shipped within 30 days of order placement, but only include countries with more than 50 total orders. 

filtered_df = df[(df['ORDERDATE'].dt.year == 2003) & (df['ORDERDATE'].dt.month <= 6)]

country_shipped_fast = filtered_df[(filtered_df['STATUS'] == 'Shipped') & (df['ORDERDATE'].dt.dayofyear <= 30)]
answer = country_shipped_fast[(country_shipped_fast['QUANTITYORDERED'] >= 50)]
print(answer[['COUNTRY', 'QUANTITYORDERED']])




    COUNTRY  QUANTITYORDERED
680     USA               50


C:\Users\Admin\AppData\Local\Temp\ipykernel_1160\2895945094.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  country_shipped_fast = filtered_df[(filtered_df['STATUS'] == 'Shipped') & (df['ORDERDATE'].dt.dayofyear <= 30)]


In [ ]:
#SQL 8:
